In [78]:
import pandas as pd
df = pd.read_json('https://data.seattle.gov/resource/47yq-6ugv.json?$limit=30000&$offset=30001&$$app_token=GphdTjjI7akvPvXuiP95RpyI3')

In [ ]:
loc_names = []
limit = 3000
offset = 3000
token = GphdTjjI7akvPvXuiP95RpyI3



In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26279 entries, 0 to 26278
Data columns (total 4 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   date                                      26279 non-null  datetime64[ns]
 1   nw_58th_st_greenway_st_22nd_ave_nw_total  26276 non-null  float64       
 2   east                                      26276 non-null  float64       
 3   west                                      26276 non-null  float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 821.3 KB


In [80]:
df['month'] = pd.DatetimeIndex(df['date']).month
df['year'] = pd.DatetimeIndex(df['date']).year
df['dow'] = pd.DatetimeIndex(df['date']).dayofweek
df['hour'] = pd.DatetimeIndex(df['date']).hour
df['am_commuter'] = df['dow'].isin([7,1,2,3,4]) & (df['hour'].isin([5,6,7,8,9]))
am_commuter_group = (df[df['am_commuter'] == True])
am_commuter_group


,date,nw_58th_st_greenway_st_22nd_ave_nw_total,east,west,month,year,dow,hour,am_commuter
52,2017-06-06 05:00:00,2.0,1.0,1.0,6,2017,1,5,True
53,2017-06-06 06:00:00,8.0,4.0,4.0,6,2017,1,6,True
54,2017-06-06 07:00:00,10.0,7.0,3.0,6,2017,1,7,True
55,2017-06-06 08:00:00,9.0,5.0,4.0,6,2017,1,8,True
56,2017-06-06 09:00:00,10.0,8.0,2.0,6,2017,1,9,True
...,...,...,...,...,...,...,...,...,...
26260,2020-06-30 05:00:00,0.0,0.0,0.0,6,2020,1,5,True
26261,2020-06-30 06:00:00,1.0,0.0,1.0,6,2020,1,6,True
26262,2020-06-30 07:00:00,3.0,3.0,0.0,6,2020,1,7,True
26263,2020-06-30 08:00:00,6.0,3.0,3.0,6,2020,1,8,True


In [82]:
df.groupby(df[['date', 'commuter']])

KeyError: "['commuter'] not in index"

In [3]:
df2 = pd.read_json('https://data.seattle.gov/resource/j4vh-b42a.json')

In [5]:
df2


,date,broadway_cycle_track_north_of_e_union_st_total,nb,sb
0,2014-01-01 00:00:00,2,0,2
1,2014-01-01 01:00:00,13,3,10
2,2014-01-01 02:00:00,2,0,2
3,2014-01-01 03:00:00,0,0,0
4,2014-01-01 04:00:00,0,0,0
...,...,...,...,...
995,2014-02-11 11:00:00,3,3,0
996,2014-02-11 12:00:00,4,0,4
997,2014-02-11 13:00:00,15,8,7
998,2014-02-11 14:00:00,14,11,3
